In [6]:
import numpy as np
import pandas as pd

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
#warnings.filterwarnings('once')
warnings.filterwarnings('ignore')

In [7]:
# Data Loading

df = pd.read_csv('winequality-red.csv')
 
df.head()



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
# Splitting training and test datasets
import tensorflow as tf

# 75% of the data is selected
train_df = df.sample(frac=0.75, random_state=4) 

# it drops the training data
# from the original dataframe
val_df = df.drop(train_df.index)


In [9]:
# Normalization

# calling to (0,1) range
max_val = train_df.max(axis= 0)
min_val = train_df.min(axis= 0)

range = max_val - min_val
train_df = (train_df - min_val)/(range)

val_df = (val_df- min_val)/range


In [10]:
# now let's separate the targets and labels
X_train = train_df.drop('quality',axis=1)
X_val = val_df.drop('quality',axis=1)
y_train = train_df['quality']
y_val = val_df['quality']

# We'll need to pass the shape
# of features/inputs as an argument
# in our model, so let's define a variable 
# to save it.
input_shape = [X_train.shape[1]]

input_shape  # number of features



[11]

In [11]:
y_train = tf.keras.utils.to_categorical(y_train-3, num_classes=6)

In [12]:
y_train

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [20]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Dropout, BatchNormalization
from keras import regularizers


In [22]:
model = Sequential()

# Add layers to the model
model.add(Dense(4096, input_shape= (X_train.shape[1],), activation='relu')) #input layer with 64 neurons
model.add(Dropout(0.5))
model.add(Dense(1024,activation= 'relu'))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(256,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(6, activation='softmax')) #output layer with 1 neuron 
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Nadam(4e-4),metrics ='Precision')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 4096)              49152     
_________________________________________________________________
dropout_6 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              4195328   
_________________________________________________________________
dense_14 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)              

In [23]:
model.fit(X_train,y_train,validation_split = 0.26,batch_size =128,epochs = 60)#,callbacks=[callbacks])

Epoch 1/60
7/7 [==============================] - 2s 112ms/step - loss: 1.1394 - precision: 1.0000 - val_loss: 0.1245 - val_precision: 0.9872
Epoch 2/60
7/7 [==============================] - 1s 75ms/step - loss: 0.0682 - precision: 0.9955 - val_loss: 0.1120 - val_precision: 0.9872
Epoch 3/60
7/7 [==============================] - 1s 74ms/step - loss: 0.0547 - precision: 0.9955 - val_loss: 0.1184 - val_precision: 0.9872
Epoch 4/60
7/7 [==============================] - 0s 69ms/step - loss: 0.0558 - precision: 0.9955 - val_loss: 0.1032 - val_precision: 0.9872
Epoch 5/60
7/7 [==============================] - 0s 68ms/step - loss: 0.0460 - precision: 0.9955 - val_loss: 0.0877 - val_precision: 0.9872
Epoch 6/60
7/7 [==============================] - 0s 73ms/step - loss: 0.0374 - precision: 0.9955 - val_loss: 0.0801 - val_precision: 0.9872
Epoch 7/60
7/7 [==============================] - 1s 84ms/step - loss: 0.0381 - precision: 0.9955 - val_loss: 0.0796 - val_precision: 0.9872
Epoch 8/60
7

Epoch 59/60
7/7 [==============================] - 1s 82ms/step - loss: 0.0161 - precision: 0.9966 - val_loss: 0.0995 - val_precision: 0.9872
Epoch 60/60
7/7 [==============================] - 1s 81ms/step - loss: 0.0172 - precision: 0.9966 - val_loss: 0.0715 - val_precision: 0.9872


In [24]:
# Get the weights of the first layer
weights = model.layers[0].get_weights()[0]

# Get the absolute values of the weights
importances = np.abs(weights)

# Normalize the importances
importances = importances / importances.sum(axis=0)

# Print the importances of each feature
for i, importance in enumerate(importances):
    print("Feature", i, "Importance", np.median(importance))

Feature 0 Importance 0.087237686
Feature 1 Importance 0.08464585
Feature 2 Importance 0.09019297
Feature 3 Importance 0.08756355
Feature 4 Importance 0.08763155
Feature 5 Importance 0.08847035
Feature 6 Importance 0.096224725
Feature 7 Importance 0.082624905
Feature 8 Importance 0.08724287
Feature 9 Importance 0.08879922
Feature 10 Importance 0.09277625


In [26]:
prediction_train = tf.argmax( model.predict(X_train), axis=1) 
print(y_pred_train)

tf.Tensor([4 4 4 ... 4 4 4], shape=(1199,), dtype=int64)


In [29]:
prediction_test = tf.argmax( model.predict(X_val), axis=1) 
print(prediction_test)

tf.Tensor(
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4], shape=(400,), dtype=int64)


In [32]:
from sklearn.metrics import plot_confusion_matrix

fig = plot_confusion_matrix(model, X_train, y_train, display_labels=model.classes_)
fig.figure_.suptitle("Confusion Matrix for Winequality Dataset - TRAIN")
plt.show()

AttributeError: 'Sequential' object has no attribute 'classes_'

In [ ]:
# https://www.kaggle.com/code/ashishkumarak/wine-quality-prediction-tensorflow

In [25]:
from kerastuner import RandomSearch
import kerastuner


def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units', min_value=2048, max_value=8116, step=512), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(units=hp.Int('units', min_value=2048, max_value=4096, step=512), activation='relu')) 
    model.add(Dropout(0.5))
    model.add(Dense(units=hp.Int('units', min_value=512, max_value=1024, step=256),activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=hp.Int('units', min_value=256, max_value=512, step=64),activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256,activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128,activation = 'relu'))
    model.add(Dense(6, activation='softmax')) 
    model.compile(optimizer=tf.keras.optimizers.Nadam(hp.Choice('learning_rate', values=[1e-3, 4e-4, 7e-4,3e-5])),
                  loss=tfa.losses.SigmoidFocalCrossEntropy(), metrics=[tfa.metrics.CohenKappa(num_classes = 6,weightage = 'quadratic')])
    return model

tuner = RandomSearch(build_model,    objective=kerastuner.Objective("val_loss", direction="min"),
     max_trials=10, project_name='intro_to_kt')
tuner.search(x=X_train, y=y_train, epochs=25,batch_size = 128, validation_split = 0.26)

best_model = tuner.get_best_models(num_models=1)[0]

ModuleNotFoundError: No module named 'kerastuner'

In [6]:
#Model building

model = tf.keras.Sequential([

    tf.keras.layers.Dense(units=64, activation='relu',input_shape=input_shape),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])


# after you create your model it's
# always a good habit to print out it's summary
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                768       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,993
Trainable params: 4,993
Non-trainable params: 0
_________________________________________________________________


In [7]:
# adam optimizer works pretty well for
# all kinds of problems and is a good starting point
model.compile(optimizer='adam', 
            # MAE error is good for numerical predictions
            loss='mae') 


In [8]:
losses = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                # it will use 'batch_size' number
                # of examples per example
                batch_size=256, 
                epochs=15, # total epoch
                )


Epoch 1/15
5/5 [==============================] - 0s 29ms/step - loss: 0.7232 - val_loss: 0.5812
Epoch 2/15
5/5 [==============================] - 0s 4ms/step - loss: 0.5115 - val_loss: 0.3969
Epoch 3/15
5/5 [==============================] - 0s 6ms/step - loss: 0.3357 - val_loss: 0.2380
Epoch 4/15
5/5 [==============================] - 0s 8ms/step - loss: 0.1893 - val_loss: 0.1400
Epoch 5/15
5/5 [==============================] - 0s 8ms/step - loss: 0.1410 - val_loss: 0.1464
Epoch 6/15
5/5 [==============================] - 0s 9ms/step - loss: 0.1539 - val_loss: 0.1474
Epoch 7/15
5/5 [==============================] - 0s 5ms/step - loss: 0.1436 - val_loss: 0.1306
Epoch 8/15
5/5 [==============================] - 0s 4ms/step - loss: 0.1274 - val_loss: 0.1210
Epoch 9/15
5/5 [==============================] - 0s 4ms/step - loss: 0.1203 - val_loss: 0.1162
Epoch 10/15
5/5 [==============================] - 0s 8ms/step - loss: 0.1166 - val_loss: 0.1120
Epoch 11/15
5/5 [=====================

In [9]:
# this will pass the first 3 rows of features
# of our data as input to make predictions
model.predict(X_val.iloc[0:3, :])



array([[0.42174911],
       [0.519015  ],
       [0.4368481 ]], dtype=float32)

In [10]:
y_val.iloc[0:3]



0     0.4
9     0.4
12    0.4
Name: quality, dtype: float64

In [12]:
loss_df

,loss,val_loss
0,0.723186,0.581182
1,0.511549,0.396933
2,0.335691,0.237998
3,0.189284,0.139953
4,0.140986,0.146401
5,0.153920,0.147429
6,0.143588,0.130583
7,0.127368,0.121019
8,0.120343,0.116173
9,0.116616,0.112020
